In [ ]:
%matplotlib inline

import ROOT, sys, os
from ROOT import std

from larcv import larcv
from larlite import larlite as ll
from larlite import larutil as lu

import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.path as path
import matplotlib.patches as patches

from ROOT import geo2d,cv
from ROOT.cv import Point_ as Vector
DTYPE='float'

import root_numpy as rn
import pandas as pd

geoh=lu.GeometryHelper.GetME()
geo=lu.Geometry.GetME()
larp=lu.LArProperties.GetME()
pygeo = geo2d.PyDraw()

matplotlib.rcParams['font.size']=20
matplotlib.rcParams['font.family']='serif'

In [ ]:
reco_event_df = pd.DataFrame(rn.root2array("../reco_ana.root",treename='EventTree'))
reco_vtx3d_df = pd.DataFrame(rn.root2array("../reco_ana.root",treename='Vtx3DTree'))
reco_particle_df = pd.DataFrame(rn.root2array("../reco_ana.root",treename='ParticleTree'))
mc_df = pd.DataFrame(rn.root2array("../mc_ana.root",treename='mctree'))

In [ ]:
mc_df.columns

In [ ]:
#Calculating the signal denomenator...
pdg2mass_dep={}
pdg2mass_dep[13]  = 105
pdg2mass_dep[2212]= 0#938
pdg2mass_dep[2112]= 0#939.5
pdg2mass_dep[11]  = 0#0.5
pdg2mass_dep[311] = 0#497.65

pdg2mass_init={}
pdg2mass_init[13]  = 0#105
pdg2mass_init[2212]= 938
pdg2mass_init[2112]= 939.5
pdg2mass_init[11]  = 0#0.5
pdg2mass_init[311] = 497.65

def mass(pdg):
    return pdg2mass_init[pdg]

mass_vec=np.vectorize(mass)

mc_wdf=mc_df.copy()
print "Total MC events:\t",mc_wdf.index.size
mc_wdf=mc_wdf.query('energyInit > 100 & energyInit < 500')
print "Events 100 and 500 MeV:\t",mc_wdf.index.size
#mc_wdf=mc_wdf[mc_wdf.daughterPdg_v.apply(lambda x : x.size>=2)]

def containment(row):
    
    pdg_v   = row['daughterPdg_v']
    
    track_id_v        = row['daughterTrackid_v']
    parent_track_id_v = row['daughterParentTrackid_v']
    
    einit_v = row['daughterEnergyInit_v']
    edep_v  = row['daughterEnergyDep_v']
    
    dep_sum=0
    init_sum=0
    #print ""
    for idx in np.where(track_id_v==parent_track_id_v)[0]:
        #print "Next primary... is PDG: ", pdg_v[idx]
        my_track_id = track_id_v[idx]
        for track_idx in np.where(my_track_id == parent_track_id_v)[0]:
            this_track_id=parent_track_id_v[track_idx]
            this_pdg=pdg_v[track_idx]
            #print "this_pdg: ",this_pdg,"... this_track_id: ",this_track_id
            #print "edep_v:   ",edep_v[track_idx], "... pdg2mass_dep",pdg2mass_dep[this_pdg]
            #print "einit_v:  ",einit_v[track_idx],"... pdg2mass_init",pdg2mass_init[this_pdg]
            dep_sum  += edep_v[track_idx]  + pdg2mass_dep[this_pdg]
            init_sum += einit_v[track_idx] - pdg2mass_init[this_pdg]
    if np.where(track_id_v==parent_track_id_v)[0].size==0:
        for einit,edep,pdg in zip(einit_v,edep_v,pdg_v):
            dep_sum  += edep+pdg2mass_dep[pdg]
            init_sum += einit-pdg2mass_init[pdg]
    
    #print "\n\n"
    #eisub_v=einit_v-mass_vec(pdg_v)
  
    #init_sum = eisub_v.sum()
    #dep_sum  = edep_v.sum()
    r=0.0
    if init_sum!=0.0:
        r = dep_sum / init_sum

    if r < 0.1:
        print row
        
    if (r > 0.9): 
        return True;
    return False;
    
mc_wdf=mc_wdf[mc_wdf.apply(containment,axis=1)]
print "Containment 0.9:\t",mc_wdf.index.size

def proton40(row):
 
    pdg_v   = row['daughterPdg_v']
    einit_v = row['daughterEnergyInit_v']
    
    eisub_v=einit_v-mass_vec(pdg_v)
  
    init_sum = eisub_v.sum()
    
    proton_v=eisub_v[np.where(pdg_v==2212)[0]]
    
    if(np.where(proton_v>40)[0].size>0): pass
    else: return False;
        
    nprimary=np.where(row['daughterTrackid_v']==row['daughterParentTrackid_v'])[0].size
    if nprimary<2: return False;
    
    return True;

def proton60(row):
 
    pdg_v   = row['daughterPdg_v']
    einit_v = row['daughterEnergyInit_v']
    
    eisub_v=einit_v-mass_vec(pdg_v)
  
    init_sum = eisub_v.sum()
    
    proton_v=eisub_v[np.where(pdg_v==2212)[0]]
    
    if(np.where(proton_v>60)[0].size>0): pass
    else: return False;
        
    nprimary=np.where(row['daughterTrackid_v']==row['daughterParentTrackid_v'])[0].size
    if nprimary<2: return False;
    
    return True;

def proton80(row):
    pdg_v   = row['daughterPdg_v']
    einit_v = row['daughterEnergyInit_v']
    
    eisub_v=einit_v-mass_vec(pdg_v)
  
    init_sum = eisub_v.sum()
    
    proton_v=eisub_v[np.where(pdg_v==2212)[0]]
    
    if(np.where(proton_v>80)[0].size>0): pass
    else: return False
    
    nprimary=np.where(row['daughterTrackid_v']==row['daughterParentTrackid_v'])[0].size
    if nprimary<2: return False;
        
    return True;
mc_wdf_40=mc_wdf[mc_wdf.apply(proton40,axis=1)]
mc_wdf_60=mc_wdf[mc_wdf.apply(proton60,axis=1)]
mc_wdf_80=mc_wdf[mc_wdf.apply(proton80,axis=1)]

print ">=2 primary & at least 1 proton min 40 MeV:\t",mc_wdf_40.index.size
print ">=2 primary & at least 1 proton min 60 MeV:\t",mc_wdf_60.index.size
print ">=2 primary & at least 1 proton min 80 MeV:\t",mc_wdf_80.index.size

mc_wdf=mc_wdf_80

In [ ]:
def lepton_angle(row):
    pdg_v=row['daughterPdg_v']
    pdg_idx_v=np.where(pdg_v==13)[0]

    if pdg_idx_v.size==0: return np.nan

    pdg_idx = pdg_idx_v[0]
    
    cos_vv=row['daughter2DCosAngle_vv']
    p0=cos_vv[0][pdg_idx]
    p1=cos_vv[1][pdg_idx]
    p2=cos_vv[2][pdg_idx]
    return [p0,p1,p2]

cos=np.row_stack(mc_wdf.apply(lepton_angle,axis=1).dropna())

for plane in xrange(3):
    plt.hist(cos[:,plane],bins=np.arange(-1.0,1.0+0.05,0.05))
    ax=plt.gca()
    ax.set_xlabel("Lepton Cos Plane {}".format(plane),fontweight='bold')
    plt.grid()
    ax.set_xlim(-1.0,1.0)
    plt.show()
plt.show()

In [ ]:
def proton_angle(row):
    pdg_v=row['daughterPdg_v']
    pdg_idx_v=np.where(pdg_v==2212)[0]

    if pdg_idx_v.size==0: return np.nan

    pdg_idx = pdg_idx_v[0]
    
    cos_vv=row['daughter2DCosAngle_vv']
    p0=cos_vv[0][pdg_idx]
    p1=cos_vv[1][pdg_idx]
    p2=cos_vv[2][pdg_idx]
    return [p0,p1,p2]

cos=np.row_stack(mc_wdf.apply(proton_angle,axis=1).dropna())

for plane in xrange(3):
    plt.hist(cos[:,plane],bins=np.arange(-1.0,1.0+0.05,0.05))
    ax=plt.gca()
    ax.set_xlabel("Proton Cos Plane {}".format(plane),fontweight='bold')
    plt.grid()
    ax.set_xlim(-1.0,1.0)
    plt.show()
plt.show()

In [ ]:
def lepton_length(row):
    pdg_v=row['daughterPdg_v']
    pdg_idx_v=np.where(pdg_v==13)[0]

    if pdg_idx_v.size==0: return np.nan

    pdg_idx = pdg_idx_v[0]
    
    cos_vv=row['daughterLength_vv']
    p0=cos_vv[0][pdg_idx]
    p1=cos_vv[1][pdg_idx]
    p2=cos_vv[2][pdg_idx]
    return [p0,p1,p2]

length_=np.row_stack(mc_wdf.apply(lepton_length,axis=1).dropna())

for plane in xrange(3):
    plt.hist(length_[:,plane],bins=np.arange(0,300+10,10))
    ax=plt.gca()
    ax.set_xlabel("Lepton Length Plane {}".format(plane),fontweight='bold')
    plt.grid()
    plt.show()
plt.show()

In [ ]:
def proton_length(row):
    pdg_v=row['daughterPdg_v']
    pdg_idx_v=np.where(pdg_v==2212)[0]

    if pdg_idx_v.size==0: return np.nan

    pdg_idx = pdg_idx_v[0]
    
    cos_vv=row['daughterLength_vv']
    p0=cos_vv[0][pdg_idx]
    p1=cos_vv[1][pdg_idx]
    p2=cos_vv[2][pdg_idx]
    return [p0,p1,p2]

length_=np.row_stack(mc_wdf.apply(proton_length,axis=1).dropna())

for plane in xrange(3):
    plt.hist(length_[:,plane],bins=np.arange(0,150+5,5))
    ax=plt.gca()
    ax.set_xlabel("Proton Length Plane {}".format(plane),fontweight='bold')
    ax.set_xlim(0,150)
    plt.grid()
    plt.show()


In [ ]:
def proton_energy_init(row):
    pdg_v=row['daughterPdg_v']
    pdg_idx_v=np.where(pdg_v==2212)[0]

    if pdg_idx_v.size==0: return np.nan

    pdg_idx = pdg_idx_v[0]
    
    e=row['daughterEnergyInit_v'][pdg_idx]-938.0
    return e
energy_=mc_wdf_40.apply(proton_energy_init,axis=1).dropna()
plt.hist(energy_.values,np.arange(0,300+10,10))
ax=plt.gca()
ax.set_xlabel("Proton Energy [MeV]",fontweight='bold')
ax.set_title("Proton E > 40",fontweight='bold')
ax.set_xlim(0,300)
ax.set_ylim(0,25)
plt.grid()
plt.show()
energy_=mc_wdf_60.apply(proton_energy_init,axis=1).dropna()
plt.hist(energy_.values,np.arange(0,300+10,10))
ax=plt.gca()
ax.set_xlabel("Proton Energy [MeV]",fontweight='bold')
ax.set_title("Proton E > 60",fontweight='bold')
ax.set_xlim(0,300)
ax.set_ylim(0,25)
plt.grid()
plt.show()
energy_=mc_wdf_80.apply(proton_energy_init,axis=1).dropna()
plt.hist(energy_.values,np.arange(0,300+10,10))
ax=plt.gca()
ax.set_xlabel("Proton Energy [MeV]",fontweight='bold')
ax.set_title("Proton E > 80",fontweight='bold')
ax.set_xlim(0,300)
ax.set_ylim(0,25)
plt.grid()
plt.show()

In [ ]:
def lepton_energy_init(row):
    pdg_v=row['daughterPdg_v']
    pdg_idx_v=np.where(pdg_v==2212)[0]

    if pdg_idx_v.size==0: return np.nan

    pdg_idx = pdg_idx_v[0]
    
    e=row['daughterEnergyInit_v'][pdg_idx]
    return e

energy_=mc_wdf.apply(lepton_energy_init,axis=1).dropna()
plt.hist(energy_.values,np.arange(0,1250+10,10))
ax=plt.gca()
ax.set_xlabel("Lepton Energy [MeV]",fontweight='bold')
plt.grid()
plt.show()

In [ ]:
#Calculating the efficiency numerator
#reco_df_copy will have our analysis cuts
reco_event_wdf = reco_event_df.ix[mc_wdf_80.index].copy()

cuts_v=[]
cuts_v.append('n_vtx3d>0')

for cut in cuts_v:
    reco_event_wdf = reco_event_wdf.query(cut)

print reco_event_wdf.index.size

In [ ]:
###############IGNORE...................###################
#join the event and vtx3d dataframes
#the event data frame has been filtered by number of 3D vertex
#a=reco_event_wdf.set_index(['run','subrun','event'])
#b=reco_vtx3d_df.set_index(['run','subrun','event'])

#get the vtx3d entries that are in events with 3D vertex
#reco_df=a.join(b)

#group the event,run,and subrun into groups (1 group == 1 event now...)
#event_group=reco_df.groupby(reco_df.index)

#for each group look at the num pixel frac, return the index in the group that gives highest SUM! -- this is vtx id
#g=event_group['num_pixel_frac_v'].apply(lambda x : np.argmax(np.row_stack(x.values).sum(axis=1)))

#top_idx=np.concatenate((np.vstack(g.index.values),g.values.reshape(g.values.shape[0],1)),axis=1)

#set the vertex ID on the index, this allows us to selection the correct r/s/e/vtxid
#reco_df=reco_df.set_index('vtx3d_id',append=True,drop=True)

#get only the index if the highest pixel fraction
#reco_df=reco_df.ix[pd.Index(top_idx).astype(np.object)]

#chosen reco dataframe (after analysis cuts)
#reco_cdf = reco_df.reset_index(['vtx3d_id'])

In [ ]:
mc_zdf=mc_wdf_80.set_index(['run','subrun','event'])
reco_event_zdf=reco_event_df.set_index(['run','subrun','event'])
reco_vtx3d_zdf=reco_vtx3d_df.set_index(['run','subrun','event'])
reco_par_zdf=reco_particle_df.set_index(['run','subrun','event'])


n_event_vtx3d=0
n_event_vtx3d_f=0
n_event_vtx3d_f_a=0

cc0=[]
cc1=[]

n_single_choice=0
n_single_choice_right_good=0
n_single_choice_right_bad=0
n_single_choice_wrong_good=0
n_single_choice_wrong_bad=0
n_make_a_choice=0
n_reco_chose_wrong=0
n_reco_chose_wrong_good=0
n_reco_chose_wrong_bad=0
n_reco_chose_right=0
n_reco_chose_right_good=0
n_reco_chose_right_bad=0
n_filtered=0

n_1_vtx=0
n_2_vtx=0
n_3_vtx=0
n_4more_vtx=0

n_good_reco=0
n_no_good_reco=0

groups_=mc_zdf.groupby(mc_zdf.index)
n_total=len(groups_)

cc0=[]
cc1=[]
def correct(reco_vtx3d_,group_,chosen_idx_):

    recox = reco_vtx3d_.vtx2d_x_v.values[chosen_idx_]
    recoy = reco_vtx3d_.vtx2d_y_v.values[chosen_idx_]
    mcx   = group_.vtx2d_t.values[0]
    mcy   = group_.vtx2d_w.values[0]

    return np.sqrt(np.power(recox-mcx,2)+np.power(recoy-mcy,2)).sum() < 7*3#6*3



DEBUG=True
for name, group in groups_:
    
    b_single_choice=False
    b_single_choice_wrong=False
    b_single_choice_right=False
    b_reco_chose_wrong=False
    b_reco_chose_right=False
    b_good_reco=False
    b_no_good_reco=False
    b_make_a_choice=False
    
    query_='run=={}&subrun=={}&event=={}'.format(name[0],name[1],name[2])
    reco_event = reco_event_df.query(query_)

    if DEBUG: print "run: ",name[0],"... subrun: ",name[1],"... event: ",name[2]
    
    nvtx=reco_event['n_vtx3d'].values[0]

    if nvtx==0: 
        continue

    n_event_vtx3d+=1;
    
    #pre filtering of n_pix_frac 
    
    reco_vtx3d = reco_vtx3d_df.query(query_)
    reco_par   = reco_particle_df.query(query_)

    #type_v = reco_vtx3d.vtx3d_type.values.tolist()
    type_v = reco_vtx3d.vtx3d_type.values
    #print type_v,np.where(type_v==0L)[0].size
    #if nvtx==1 or (nvtx==2 and ((0 in type_v) and (1 in type_v))):
    if nvtx==1 or (nvtx>1 and np.where(type_v==0L)[0].size==1):
        if DEBUG: print "Single choice"
            
        b_single_choice=True
        type_v=np.array(type_v)
        if nvtx==2:
            chosen_idx = np.where(type_v==0L)[0][0]
        else:
            chosen_idx=0
        
        close = correct(reco_vtx3d,group,chosen_idx)

        if close == True: 
            b_single_choice_right=True
        else: 
            b_single_choice_wrong=True
        
    else:
        
        n_pixel_frac_v=reco_vtx3d.num_pixel_frac_v

        bad_idx=[]
        for ix,npf in enumerate(n_pixel_frac_v.values):
            if ( np.where(npf>0.8)[0].size==3 ): continue;
            bad_idx.append(ix)

        reco_vtx3d = reco_vtx3d.query('vtx3d_id not in @bad_idx')
        reco_par   = reco_par.query  ('vtx3d_id not in @bad_idx')

        nvtx2=reco_vtx3d.index.values.size
        if nvtx2 < nvtx:
            if DEBUG: 
                print "Filtered NPX frac"
        nvtx=nvtx2
        
        
        
        if nvtx==0: 
            if DEBUG: print "Filtered NPX frac bad_idx: ",bad_idx
            n_event_vtx3d_f+=1;
            continue
    
        angle_vv=reco_vtx3d.circle_vtx_angle_v
        
        bad_idx=[]
        for ix,angle_v in enumerate(angle_vv.values):
            if( np.where(angle_v<170.)[0].size>1 ): continue;
            bad_idx.append(ix)
        
        reco_vtx3d = reco_vtx3d.query('vtx3d_id not in @bad_idx')
        reco_par   = reco_par.query  ('vtx3d_id not in @bad_idx')

        nvtx2=reco_vtx3d.index.values.size
        if nvtx2 < nvtx:
            if DEBUG: 
                print "Filtered angle_vv bad_idx: ",bad_idx
        nvtx=nvtx2
        if nvtx==0:
            n_event_vtx3d_f_a+=1;
            continue   
            
        b_make_a_choice=True
        # make your choice...
        #chosen_idx=np.argmax(np.row_stack(reco_vtx3d.circle_vtx_r_v.values).sum(axis=1))
        if nvtx==1:
            n_1_vtx+=1
        elif nvtx==2:
            n_2_vtx+=1
        elif nvtx==3:
            n_3_vtx+=1
        else:
            n_4more_vtx+=1
           
        sum_pix_frac_copy = reco_vtx3d.sum_pixel_frac.values.tolist()

        angle_sum_v = [0.]*len(sum_pix_frac_copy)
        
        for ix,angle_v in enumerate(reco_vtx3d.circle_vtx_angle_v.values):
            angle_sum_v[ix] = angle_v.sum()

        #angle_sum_v=np.array(angle_sum_v)
        #chosen_idx=np.argmin(angle_sum_v)
    
        #make the decision based on the length of th shortest track
            
        dec_=[]
        print "reco vtx3d id values: ",reco_vtx3d.vtx3d_id.values
        
        for ix,vtx_id in enumerate(reco_vtx3d.vtx3d_id.values):
            
            dec_.append(0)
            
            thisvtx=reco_par.query('vtx3d_id=={}'.format(vtx_id))
    
            for plane_id in xrange(3):
                thisplane=thisvtx.query("plane_id=={}".format(plane_id))

                if (thisplane.n_pars.values[0] < 2): 
                    continue
                    
                #get the shortest track

                s_idx=np.argmin(thisplane.atom_sum_length_v.values[0])
                
                #get the direction of the shortest track
                sy=thisplane.start_y_v.values[0][s_idx]
                ey=thisplane.end_y_v  .values[0][s_idx]                    

                if (ey>sy):
                    dec_[-1]+=1
        
        
        dec_=np.array(dec_)
        
        chosen_idx=np.argmax(dec_)
        print "chosen_idx: ", chosen_idx, "... dec_ ",dec_
        print ""
        close = correct(reco_vtx3d,group,chosen_idx)

        if close == True: 
            b_reco_chose_right=True
        else: 
            b_reco_chose_wrong=True


    #is there a reco vertex, we just chose it wrong?
    good_reco=False
    for idx_ in xrange(nvtx):
        
        if good_reco==True : continue
        good_reco = correct(reco_vtx3d,group,idx_)
        
    if good_reco==True: 
        b_good_reco=True
    else:
        b_no_good_reco=True

    if b_single_choice==True:
        n_single_choice+=1
        
    if b_single_choice_right==True and b_no_good_reco==False:
        n_single_choice_right_good+=1
         
    if b_single_choice_right==True and b_no_good_reco==True:
        n_single_choice_right_bad+=1
        
    if b_single_choice_wrong==True and b_no_good_reco==False:
        n_single_choice_wrong_good+=1
        
    if b_single_choice_wrong==True and b_no_good_reco==True:
        n_single_choice_wrong_bad+=1
        
    if b_make_a_choice==True:
        n_make_a_choice+=1
        
    if b_reco_chose_right==True:
        n_reco_chose_right+=1
        
    if b_reco_chose_right==True and b_no_good_reco==False:
        n_reco_chose_right_good+=1
        cc0.append(mc_df.query(query_).index.values[0])
        cc1.append(reco_par_zdf.query(query_).index.values[0])
        
    if b_reco_chose_right==True and b_no_good_reco==True:
        n_reco_chose_right_bad+=1
        
    if b_reco_chose_wrong==True:
        n_reco_chose_wrong+=1
  
    if b_reco_chose_wrong==True and b_no_good_reco==False:
        n_reco_chose_wrong_good+=1

       
    if b_reco_chose_wrong==True and b_no_good_reco==True:
        n_reco_chose_wrong_bad+=1
    
    if b_reco_chose_right == True and b_reco_chose_wrong == True:
        assert(False)
    
    if b_good_reco==True:
        n_good_reco+=1
    if b_no_good_reco==True:
        n_no_good_reco+=1


    
print "n_total:\t",n_total
print "\nn_event_vtx3d:\t\t",n_event_vtx3d
print "n_event_vtx3d_f_n_pix_frac\t",n_event_vtx3d_f
print "n_event_vtx3d_f_angle:\t",n_event_vtx3d_f_a
print "\nn_events_parsed:\t",n_event_vtx3d-n_event_vtx3d_f-n_event_vtx3d_f_a
print "n_1_vtx:\t",n_1_vtx
print "n_2_vtx:\t",n_2_vtx
print "n_3_vtx:\t",n_3_vtx
print "n_4more_vtx:\t",n_4more_vtx

print "\nn_single_choice:\t",n_single_choice
print "n_single_choice_right_good:",n_single_choice_right_good
print "n_single_choice_right_bad: ",n_single_choice_right_bad
print "n_single_choice_wrong_good:",n_single_choice_wrong_good
print "n_single_choice_wrong_bad: ",n_single_choice_wrong_bad
print "\nn_make_a_choice:       ",n_make_a_choice
print "n_reco_chose_right:      ",n_reco_chose_right
print "n_reco_chose_right_good: ",n_reco_chose_right_good
print "n_reco_chose_right_bad:  ",n_reco_chose_right_bad
print "n_reco_chose_wrong:      ",n_reco_chose_wrong
print "n_reco_chose_wrong_good: ",n_reco_chose_wrong_good
print "n_reco_chose_wrong_bad:  ",n_reco_chose_wrong_bad

print "\ngood possible vtx diagnostic"
print "total reco correct:\t",n_single_choice_right_good+n_reco_chose_right_good
print "total reco wrong:\t",n_single_choice_wrong_good+n_reco_chose_wrong_good

print "\nbest case senario"
print "n_good_reco:\t\t",n_good_reco
print "n_no_good_reco:\t\t",n_no_good_reco
print "\n\n"
print cc0
print cc1

In [ ]:
def rolling_mean(arr,pre,post,frac_below=0.0,frac_above=1.0):
    res=[-1.0] * len(arr)
    for x in xrange(len(arr)):
        if arr[x] == -1: continue
        if x+1    <  pre: continue
        if x+post >= len(arr): break
        partarray = np.array(arr[(x+1-pre):x+post])
        partarray = partarray[partarray>=0]
        if len(partarray)<1: continue
        sort_partarray = np.sort(partarray)
        nn=sort_partarray.size
        res[x]=sort_partarray[np.ceil(frac_below*nn):np.floor(frac_above*nn)].mean()
    return res
def rolling_sigma(arr,pre,post,frac_below=0.0,frac_above=1.0):
    res=[-1.0] * len(arr)
    for x in xrange(len(arr)):
        if arr[x] == -1: continue
        if x+1    <  pre: continue
        if x+post >= len(arr): break
        partarray = np.array(arr[(x+1-pre):x+post])
        partarray = partarray[partarray>=0]
        if len(partarray)<1: continue
        sort_partarray = np.sort(partarray)
        nn=sort_partarray.size
        res[x]=sort_partarray[np.ceil(frac_below*nn):np.floor(frac_above*nn)].std()
    return res

In [ ]:
class Pulse:
    def __init__(self,tstart,tend,tmax,amp,area,ped,sig):
        self.tstart= tstart
        self.tend  = tend
        self.tmax  = tmax
        self.amp   = amp
        self.area  = area
        self.ped   = ped
        self.sig   = sig
        
    def dt(self):
        return tmax-tend

    def f(self):
        return (self.amp-self.ped) / float(self.sig)
    
def search_pulses(rmean,f0,f1,samples,baseline_mean,baseline_sigma):
    pulses=[]
    found=False
    for ix in xrange(len(rmean)):
        if ix<samples:            
            continue
        if ix>len(rmean)-samples:
            continue

        adc=rmean[ix]

        #pulse found
        if (adc > (baseline_mean + f0*baseline_sigma)):
            if found == True:
                continue

            tstart=ix
            padc=rmean[tstart]
            while (padc > (baseline_mean+f1*baseline_sigma)):
                tstart -= 1
                padc    = rmean[tstart]

            tend=tstart+1  
            padc=rmean[tend]

            while (padc > (baseline_mean+f1*baseline_sigma)):
                tend += 1
                padc = rmean[tend]

            tmax=np.argmax(rmean[tstart:tend])+tstart

            #print "Found pulse @ tstart: ",tstart," tend: ",tend," tmax: ",tmax," at adc: ", rmean[tmax]
            found=True
            pulses.append(Pulse(tstart,tend,tmax,rmean[tmax],-1.0,baseline_mean,baseline_sigma))

            continue

        found=False
    
    pulses_f=[]
    
    for i in xrange(len(pulses)):
        pulse_i = pulses[i]
        dupe=False
        for j in xrange(i+1,len(pulses)):
            pulse_j = pulses[j]
            if pulse_i.tstart == pulse_j.tstart:
                dupe=True
        if dupe==False:
            pulses_f.append(pulse_i)
    
    #for p in pulses_f:
    #    print "Found pulse @ tstart: ",p.tstart," tend: ",p.tend," tmax: ",p.tmax," at adc: ", p.amp," f: ",p.f()
    return pulses_f
            
this_df=reco_par_zdf.ix[(4, 790, 15796)]
for plane in xrange(3):
    for vtxid in xrange(1):
        print "++++++++++"
        abcs   = this_df.query('vtx3d_id==@vtxid').query('plane_id==@plane').dqdx_vv.values[0]
        abcidx = this_df.query('vtx3d_id==@vtxid').query('plane_id==@plane').dqdx_start_idx_vv.values[0]
        
        #sizes=np.array([abc.size for abc in abcs])
        #ab=abcs[np.argmax(sizes)]
        
        for ix,ab in enumerate(abcs):
            #asidx=abcidx[ix]
            plt.plot(ab,'-o',color='blue')
            rmean  = rolling_mean (ab,2,2)
            rsigma = rolling_sigma(ab,2,2)
            plt.plot(rmean,'-',color='red',lw=2)
            ax=plt.gca()

            hist,_=np.histogram(rmean,bins=100,range=(0,1000))
            baseline_mean=np.argmax(hist)*10+5
            hist,_=np.histogram(rsigma,bins=100,range=(0,100))
            baseline_sigma=np.argmax(hist)*1+0.5
            print "mean, sigma: ",baseline_mean,baseline_sigma

            #find the pulses
            f0=5.0
            f1=1.0
            pulses=search_pulses(rmean,f0,f1,2,baseline_mean,baseline_sigma)

            ax.hlines(baseline_mean,0,len(ab),lw=3,color='purple',alpha=0.75)
            ax.hlines(baseline_mean+f0*baseline_sigma,0,len(ab),lw=3,color='green',alpha=0.75)
            #ax.vlines(asidx,0,600,lw=2,color='orange',alpha=0.5)
            for pulse in pulses:
                ax.axvspan(pulse.tstart,pulse.tend,color='orange',alpha=0.5)
                ax.vlines(pulse.tmax,0,600,lw=2,color='green',alpha=0.5)

            ax.set_title("vtxid: {}... planeid: {}... pid: {}".format(vtxid,plane,ix))
            ax.set_ylim(0,400)
            plt.grid()
            #plt.savefig("%02d_%02d_%02d.png"%(0,plane,ix))
            plt.show()

    print "=========="

this_df

In [ ]:
event_group=reco_df.groupby(reco_df.index)
#get the vertex with highest sum_pixel_frac
g=event_group['sum_pixel_frac'].apply(lambda x : np.argmax(x.values))
top_idx=np.concatenate((np.vstack(g.index.values),
                        g.values.reshape(g.values.shape[0],1)),axis=1)
reco_cdf=reco_df.set_index('vtx3d_id',append=True,drop=True)
reco_cdf=reco_cdf.ix[pd.Index(top_idx).astype(np.object)]
reco_cdf = reco_cdf.reset_index(['vtx3d_id'])

In [ ]:
mc_cdf=mc_wdf.set_index(['run','subrun','event']).ix[reco_cdf.index]

dw_v=mc_cdf.vtx2d_w.values  - reco_cdf.circle_vtx_y_v.values
dt_v=mc_cdf.vtx2d_t.values  - reco_cdf.circle_vtx_x_v.values

dw_v=np.vstack(dw_v)
dt_v=np.vstack(dt_v)

In [ ]:
#2d resolution plots 

fig,ax=plt.subplots(figsize=(10,6))
for i in xrange(3):
    ax.hist(dw_v[:,i],
            label='Plane {}'.format(i),
            bins=np.arange(-80,80+2,2),
            alpha=0.5)
ax.set_xlabel("(MC - Reco) Wire [pixel]",fontweight='bold')
ax.set_ylim(0,45)
ax.set_xticks(np.arange(-100,100,20))
ax.set_xlim(-100,100)
ax.legend()
plt.grid()
plt.show()

fig,ax=plt.subplots(figsize=(10,6))
for i in xrange(3):
    ax.hist(dt_v[:,i],
            label='Plane {}'.format(i),
            bins=np.arange(-80,80+2,2),
            alpha=0.5)
ax.set_xlabel("(MC - Reco) Time [pixel]",fontweight='bold')
ax.set_xticks(np.arange(-100,100,20))
ax.set_ylim(0,45)
ax.set_xlim(-100,100)
ax.legend()
plt.grid()
plt.show()


for i in xrange(3):
    fig,ax=plt.subplots(figsize=(10,6))
    ax.hist(np.sqrt(np.power(dw_v[:,i],2)+np.power(dt_v[:,i],2)),
            label='Plane {}'.format(i),
            bins=np.arange(0,100+1,1),
            alpha=0.5,lw=2)
    ax.set_xlabel("Dist MC Reco Vtx [pixel]",fontweight='bold')
    ax.set_ylim(0,30)
    ax.set_xlim(0,100)
    ax.legend()
    plt.grid()
    plt.show()

In [ ]:
#num_pixel_frac check
p_=reco_cdf.num_pixel_frac_v
p_=np.vstack(p_)
for i in xrange(3):
    fig,ax=plt.subplots(figsize=(10,6))
    ax.hist(p_[:,i],bins=np.arange(0,1+0.6,0.1))
    ax.set_xlabel("Plane {}".format(i))
    ax.set_ylim(0,45)
    ax.set_xlim(0,1.5)
    plt.grid()
    plt.show()

fig,ax=plt.subplots(figsize=(10,6))
ax.hist(p_.sum(axis=1),bins=np.arange(0,4+0.1,0.1))
ax.set_xlabel("Sum")
plt.grid()
plt.show()

fig,ax=plt.subplots(figsize=(10,6))
ppp=p_[:,0]*p_[:,1]*p_[:,2]
ax.hist(ppp,bins=np.arange(0,1+0.6,0.1))
ax.set_xlim(0,1.0)
ax.set_xlabel("Product")
plt.grid()
plt.show()

In [ ]:
#chose the 3D vertex with highest num_pix_frac
def hi_pix_idx(row):
    x_v = row['vtx3d_x_v']
    y_v = row['vtx3d_y_v']
    z_v = row['vtx3d_z_v']
    num_pix_frac_vv = row['num_pixel_frac_vv']
    idx=np.argmax(np.sum(np.vstack(num_pix_frac_vv),axis=1))
    return [x_v[idx],y_v[idx],z_v[idx]]

vtx_x=reco_cdf.vtx3d_x.values
vtx_y=reco_cdf.vtx3d_y.values
vtx_z=reco_cdf.vtx3d_z.values

vtx=np.vstack([vtx_x,vtx_y,vtx_z]).T

In [ ]:
dvtx=vtx-mc_cdf[['parentX','parentY','parentZ']].values

In [ ]:
fig,ax=plt.subplots(figsize=(10,6))
ax.hist(dvtx[:,0],bins=np.arange(0,100,1))
ax.set_xlabel('MC - Reco dX [pixel]',fontweight='bold')
ax.set_xlim(-100,100)
plt.grid()
plt.show()

fig,ax=plt.subplots(figsize=(10,6))
ax.hist(dvtx[:,1],bins=np.arange(-100,100,1))
ax.set_xlim(-100,100)
plt.grid()
ax.set_xlabel('MC - Reco dY [pixel]',fontweight='bold')
plt.show()

fig,ax=plt.subplots(figsize=(10,6))
ax.hist(dvtx[:,2],bins=np.arange(-100,100,1))
ax.set_xlim(-100,100)
plt.grid()
ax.set_xlabel('MC - Reco dZ [pixel]',fontweight='bold')
plt.show()

In [ ]:
nf0=[]
nf1=[]
nf2=[]
for name, row in mc_zdf.iterrows():
    muon_idx   = np.where(row['daughterPdg_v']==13)[0]
    proton_idx = np.where(row['daughterPdg_v']==2212)[0]
    
    if muon_idx.size < 1: continue
        
    muon_idx=muon_idx[0]
    proton_idx=proton_idx[0]
    
    for plane in xrange(3):
        p_sx=row.daughter2DStartX_vv[plane][proton_idx]
        p_sy=row.daughter2DStartY_vv[plane][proton_idx]
        p_ex=row.daughter2DEndX_vv[plane][proton_idx]
        p_ey=row.daughter2DEndX_vv[plane][proton_idx]
        
        m_sx=row.daughter2DStartX_vv[plane][muon_idx]
        m_sy=row.daughter2DStartY_vv[plane][muon_idx]
        m_ex=row.daughter2DEndX_vv[plane][muon_idx]
        m_ey=row.daughter2DEndX_vv[plane][muon_idx]

        p_d=np.sqrt(np.power(p_ex-p_sx,2)+np.power(p_ey-p_sy,2))
        m_d=np.sqrt(np.power(m_ex-m_sx,2)+np.power(m_ey-m_sy,2))

        if m_d > p_d :
            sx=p_sx
            ex=p_ex
        else:
            sx=m_sx
            ex=m_ex
        
        if ex>sx:
            if plane==0:nf0.append(1)
            if plane==1:nf1.append(1)
            if plane==2:nf2.append(1)
        else:
            if plane==0:nf0.append(-1)
            if plane==1:nf1.append(-1)
            if plane==2:nf2.append(-1)

nf0=np.array(nf0)
nf1=np.array(nf1)
nf2=np.array(nf2)

plt.hist(nf0,bins=np.arange(-1,3,1))
plt.grid()
ax=plt.gca()
ax.set_ylim(0,150)
ax.set_xlim(-1,2)
ax.set_title("Plane 0")
ax.set_xlabel("shortest track: -1 up +1 down")
ax.hlines(115,-1,2,color='orange',lw=2)
plt.show()
plt.hist(nf1,bins=np.arange(-1,3,1))
plt.grid()
ax=plt.gca()
ax.set_ylim(0,150)
ax.set_xlim(-1,2)
ax.set_title("Plane 1")
ax.set_xlabel("shortest track : -1 up +1 down")
ax.hlines(115,-1,2,color='orange',lw=2)
plt.show()
plt.hist(nf2,bins=np.arange(-1,3,1))
plt.grid()
ax=plt.gca()
ax.set_ylim(0,150)
ax.set_xlim(-1,2)
ax.set_title("Plane 2")
ax.set_xlabel("shortest track: -1 up +1 down")
ax.hlines(115,-1,2,color='orange',lw=2)
plt.show()


In [ ]:
mc_df.query('run==4&subrun==3&event==54')